In [1]:
# %cd ..
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import random
import os

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

device = torch.device("cuda:0") if torch.cuda.is_available() else "cpu"

In [2]:
# seed 고정
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(0)

### 평가 부분
1. 테스트를 진행하고자 하는 데이터셋을 불러온다.
2. 테스트를 진행하고자 하는 경우 논문 본문과 확인하고자 하는 하나의 타겟을 불러온다.
    - 본 실험에서는 논문 연구분야 데이터셋 30000건 중 6000건에 대해서 테스트 데이터로 활용하였다.
    - 해당 테스트 데이터는 RCMN_CD1은 nan값이 없고 그 외에는 NaN값이 존재하기에 스코어 계산에는 RCMN_CD1만 이용하였다.
    - 테스트를 진행할 경우 타겟 열 하나를 직접 지정해준다.
        - 바로 밑의 셀의 data.loc[:, ["PAPER_TEXT", 타겟 컬럼]]과 같은 방식으로 진행한다.
3. target.json에는 중분류 코드와 그에 해당하는 number가 들어가 있다. 이는 논문 연구분야 30000건에서 발췌하여 미리 저장해둔 json 파일이다.
4. 나머지 부분은 그대로 실행시키면 된다.

In [3]:
dataset = pd.read_csv("./test.csv", encoding = 'utf-8')
# 데이터셋은 논문 본문과 특정 컬럼만을 불러온다.
dataset = dataset.loc[:, ["PAPER_TEXT", "RCMN_CD1"]]
dataset.columns = ["PAPER_TEXT", "target"]
dataset

,PAPER_TEXT,target
0,서론 최근 사람의 감정을 인식하고 응용하는 분야가 많아지고 있다. 감정을 인식하고 ...,ED0701
1,서론 농어업 농어촌은 농산물 시장개방으로 인한 가격하락과 도농 간의 격차가 심화되어...,LB2006
2,INTRODUCTION The invisible inner damage of com...,EB0707
3,서론 전 세계적으로 국가 간 무역장벽이 없어지면서 글로벌 경쟁이 치열해 지고 있고 ...,SG0401
4,INTRODUCTION Developments in the ceramic mater...,EB0208
...,...,...
5995,연구의 필요성 및 목적 인간의 모든 행위는 공간 속에서 이루어지므로 공간에 대한 정...,EI0101
5996,서론 최근 한국의 고등교육기관을 찾는 외국인 유학생 수가 급증하고 있다. 교육통계서...,HC0217
5997,서론 최근 해양레저산업선진국의 경우 해양레저 활동 범위의 확대에 따라 원거리 항해에...,EA1001
5998,서론 오늘날 대한민국의 대학도서관은 모기관인 대학의 조직 및 인력 축소로 인해 사서...,SI0405


In [4]:
# target.json에는 중분류 코드와 중분류 정보가 담겨있다.
import json
with open("./target.json", "r") as f:
    d = json.load(f)

for i in range(len(dataset)):
    dataset.loc[i, "target"] = dataset.loc[i, "target"][:4]

# test 데이터프레임에 컬럼을 load한다.
t = []
for i in range(len(dataset)):
    t.append(d[dataset.loc[i, "target"]])
dataset["TARGET"] = t

In [5]:
def make_dataset(mode):
    data = dataset
    mk_data = []
    for sentence, label in tqdm(zip(data["PAPER_TEXT"], data["TARGET"])):
        # sentence_list = sentence.split(".")
        # sentences = []
        # for sentence in sentence_list:
        #     sentence = sentence.strip()
        #     if sentence != 'I' and sentence != 'II' and sentence != "" and sentence != " " and sentence != "표" and sentence != "그림":
        #         sentences.append(sentence)
        #         sentences.append(" [SEP] ")
        # sentences = sentences[:-1]
        # sentences.insert(0, "[CLS] ")
        data_ = [sentence, label]
        mk_data.append(data_)
    return mk_data

test_set = make_dataset("test")

6000it [00:00, 1211293.03it/s]


In [6]:
from gluonnlp.vocab import Vocab
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertTokenizer
from transformers import BertModel
import json
# 이를 불러오고자 한다면 return_dict는 무조건 false로 해야한다.
bertmodel = BertModel.from_pretrained("skt/kobert-base-v1", return_dict = False)
# tokenizer = KoBERTTokenizer.from_pretrained("skt/kobert-base-v1")
# 커스텀 trained vocab txt 파일을 가지고 BertTokenzier 생성
tokenizer = BertTokenizer.from_pretrained("./vocab.txt", do_basic_tokenize = False)
# vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token = "[PAD]")
tok = tokenizer.tokenize
# customize vocab을 직접 불러온다.
with open("vocab.txt", "r") as f:
    vocab = f.readlines()
vocab = [voc.rstrip("\n") for voc in vocab]
dic = dict()
for i in range(len(vocab)):
    dic[vocab[i]] = i
# 최종적으로 vocab 형태를 만들어준다.
vocab = Vocab(dic)

In [7]:
bertmodel.resize_token_embeddings(len(vocab))
max_len = 128

In [8]:
# BERTDataset 형태로 변경하는 과정
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len, pad, pair, mode = 'train'):
        transform = nlp.data.BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair, vocab = vocab)
        self.mode = mode
        if self.mode == "train":
            self.sentences = [transform([i[sent_idx]]) for i in dataset]
            self.labels = [np.int32(i[label_idx]) for i in dataset]
        else:
            self.sentences = [transform(i) for i in dataset]

    def __getitem__(self, i):
        if self.mode == "train":
            return (self.sentences[i] + (self.labels[i], ))
        else:
            return self.sentences[i]

    def __len__(self):
        return (len(self.sentences))

data_test = BERTDataset(dataset = test_set, sent_idx = 0, label_idx = 1, bert_tokenizer = tok, vocab = vocab, max_len = max_len, pad = True, pair = False, mode = "test")
test_dataloader = DataLoader(data_test, batch_size = 1, num_workers = 4)

In [9]:
class BERTClassifier(nn.Module):
    def __init__(self, bert, hidden_size = 768, num_classes=365, dr_rate=None, params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
        torch.nn.init.kaiming_uniform(self.classifier.weight)
    
    def get_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.get_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [10]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = torch.load("./model.pt")
model.to(device)

BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8005, 768)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
             

In [11]:
# 해당 함수를 통해 test 결과를 반환.
def inference():  
    model.eval()
    outputs = []
    out_logits = []
    with torch.no_grad():
        for batch_id, (token_ids, valid_length, segment_ids) in tqdm(enumerate(test_dataloader)):
            token_ids = token_ids.long().to(device)
            segment_ids = segment_ids.long().to(device)
            
            valid_length= valid_length
    
            out = model(token_ids, valid_length, segment_ids)
    
            for i in out:
                logits = i
                logits = logits.detach().cpu().numpy()
                logits = np.argmax(logits)
            outputs.append(logits)
            out_logits.append(out)

        return outputs, out_logits

In [12]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
# out 지정
out = inference()
# label 지정
labels = dataset["TARGET"]

6000it [01:07, 88.76it/s] 


In [13]:
output = out[0]
logits = out[1]
print("-------------------------------------------------")
print("accuracy score         |  {}".format(accuracy_score(output, labels)))
print("-------------------------------------------------")
print("precision macro score  |  {}".format(precision_score(output, labels, average = "macro")))
print("-------------------------------------------------")
print("precision micro score  |  {}".format(precision_score(output, labels, average = "micro")))
print("-------------------------------------------------")
print("recall macro score     |  {}".format(recall_score(output, labels, average = "macro")))
print("-------------------------------------------------")
print("recall micro score     |  {}".format(recall_score(output, labels, average = "micro")))
print("-------------------------------------------------")

-------------------------------------------------
accuracy score         |  0.5988333333333333
-------------------------------------------------
precision macro score  |  0.5054755796695363
-------------------------------------------------
precision micro score  |  0.5988333333333333
-------------------------------------------------
recall macro score     |  0.5699121308731049
-------------------------------------------------
recall micro score     |  0.5988333333333333
-------------------------------------------------


In [14]:
top_ = []
def mk_top3(tensor_data):
    t_data = tensor_data.cpu().numpy()
    top_indices = np.argsort(t_data)[-3:]
    top_values = t_data[top_indices]
    return top_indices
for outs in logits:
    top_.append(mk_top3(outs[0]))

In [15]:
# dataframe으로 보여준다.
key = list(d.keys())
top3_keys = pd.DataFrame(top_)
for i in range(len(top3_keys)):
    top3_keys.loc[i, 0] = key[top3_keys.loc[i, 0]]
    top3_keys.loc[i, 1] = key[top3_keys.loc[i, 1]]
    top3_keys.loc[i, 2] = key[top3_keys.loc[i, 2]]
top3_keys.columns = ["label1", "label2", "label3"]

In [16]:
top3_keys.to_csv("top3_keysout.csv", index = False)

In [17]:
top3_keys

,label1,label2,label3
0,EA05,NA06,ED07
1,SC04,SF02,SC06
2,EG02,EB01,EB07
3,SE05,SB12,SC07
4,EH07,EB05,EB02
...,...,...,...
5995,SF03,SF02,EI01
5996,HC01,HC05,HC02
5997,EB01,EB06,EB07
5998,SI05,SI01,SI04


### 유사 논문 추천 부분

In [18]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def find_similar_documents(new_data, top_n=10):
    # final.csv 파일에서 데이터 불러오기
    final_dataset = pd.read_csv('final.csv', encoding='utf-8')
    final_dataset = final_dataset.head(100)
    final_dataset['PAPER_TEXT'].fillna('', inplace=True)
    
    results = []

    for i in range(len(new_data)):
        # 새로운 데이터를 가져옴
        new_document = new_data[i]

        # 전처리된 텍스트를 TF-IDF 벡터로 변환
        tfidf_vectorizer = TfidfVectorizer()
        tfidf_matrix = tfidf_vectorizer.fit_transform(final_dataset['PAPER_TEXT'] + [new_document])

        # 입력 문서와 다른 문서들 간의 Cosine Similarity 계산
        cosine_similarities = cosine_similarity(tfidf_matrix[-1], tfidf_matrix[:-1])

        # 가장 유사한 문서들의 인덱스 추출
        similar_documents_indices = cosine_similarities[0].argsort()[:-top_n-1:-1]

        # 결과를 리스트에 추가
        result = []
        for idx in similar_documents_indices:
            similarity_score = cosine_similarities[0][idx]
            item = {
                "인덱스": idx,
                "코사인 유사도": similarity_score,
                #"문서 이름": final_dataset['PAPER_NM_KO'].iloc[idx],
                "중분류 코드": final_dataset['target'].iloc[idx]
            }
            result.append(item)

        results.append(result)

    return results

In [19]:
find_similar_documents(dataset['PAPER_TEXT'], top_n=3)

[[{'인덱스': 70, '코사인 유사도': 0.577162794762542, '중분류 코드': 'NB06'},
  {'인덱스': 90, '코사인 유사도': 0.5693437444023912, '중분류 코드': 'NC01'},
  {'인덱스': 91, '코사인 유사도': 0.5689949453578996, '중분류 코드': 'NC01'}],
 [{'인덱스': 56, '코사인 유사도': 0.6329701414982096, '중분류 코드': 'NB06'},
  {'인덱스': 92, '코사인 유사도': 0.6315930138753715, '중분류 코드': 'NC01'},
  {'인덱스': 70, '코사인 유사도': 0.6302168578777945, '중분류 코드': 'NB06'}],
 [{'인덱스': 70, '코사인 유사도': 0.6934839766183589, '중분류 코드': 'NB06'},
  {'인덱스': 90, '코사인 유사도': 0.6825775945458832, '중분류 코드': 'NC01'},
  {'인덱스': 56, '코사인 유사도': 0.6794659276125377, '중분류 코드': 'NB06'}],
 [{'인덱스': 94, '코사인 유사도': 0.4628313657696655, '중분류 코드': 'NC01'},
  {'인덱스': 92, '코사인 유사도': 0.4593854470786078, '중분류 코드': 'NC01'},
  {'인덱스': 56, '코사인 유사도': 0.4590057586478958, '중분류 코드': 'NB06'}],
 [{'인덱스': 70, '코사인 유사도': 0.71935236832487, '중분류 코드': 'NB06'},
  {'인덱스': 90, '코사인 유사도': 0.7114501489630135, '중분류 코드': 'NC01'},
  {'인덱스': 92, '코사인 유사도': 0.7090245094516182, '중분류 코드': 'NC01'}],
 [{'인덱스': 92, '코사인 유사도': 0.38695172776